In [1]:
import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool
from datetime import datetime

#from docx import Document
from io import BytesIO
import base64

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/Albert/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] = os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0806',temperature=0)

In [3]:
df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_NEW.csv')

/home/jovyan/tmp/ipykernel_172308/3347160452.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_NEW.csv')


In [4]:
df.head()

,ID,DOS,TYPE,CODE,VALUE,ABNORMAL_CD,CD_DESC
0,33260,2019-10-23,EMR_Note,chief complaint,pre-op evaluation; hypertension. needs cx for ...,NaN,NaN
1,33260,2019-10-23,EMR_Note,past medical history,anemia of chronic disease. Percutaneous Corona...,NaN,NaN
2,33260,2019-11-16,EMR_Note,history of present illness,Pt is here for follow up from the hospital. Pt...,NaN,NaN
3,33260,2019-11-16,EMR_Note,physical exam,Cardiology Brief Exam. Basic Cardio PE:. HEENT...,NaN,NaN
4,33260,2019-11-16,EMR_Note,instructions,I10: Essential (primary) hypertension. 5: Hype...,NaN,NaN


In [5]:
df.columns=df.columns.str.lower()

In [6]:
#df=df[~df['type'].isin(['EMR_Note'])]

In [7]:
# df['value'] = pd.to_numeric(df['value'], errors='coerce')
# df['value']=np.round(df['value'],2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061046 entries, 0 to 1061045
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           1061046 non-null  int64 
 1   dos          1056638 non-null  object
 2   type         1061046 non-null  object
 3   code         1056286 non-null  object
 4   value        176927 non-null   object
 5   abnormal_cd  125899 non-null   object
 6   cd_desc      985353 non-null   object
dtypes: int64(1), object(6)
memory usage: 56.7+ MB


In [9]:
df.columns = df.columns.str.strip()

In [10]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [11]:
# Define the new basic member information agent
basic_info_agent = Agent(
    role="Basic Member Information Provider",
    goal="Display the member ID, age, and gender for the specified member",
    backstory="You are a data retrieval specialist responsible for quickly and accurately providing basic member information.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

In [12]:
diagnosis_agent = Agent(
    role="Diagnosis Analyst",
    goal="Provide a comprehensive analysis of the patient's diagnostic history",
    backstory="You are an expert in medical diagnostics with years of experience in analyzing patient histories.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

procedure_agent = Agent(
    role="Procedure Analyst",
    goal="Analyze and summarize all medical procedures the patient has undergone",
    backstory="You are a medical procedure specialist with deep knowledge of various medical interventions.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

medication_agent = Agent(
    role="Medication Analyst",
    goal="Provide a detailed overview of the patient's medication history",
    backstory="You are a clinical pharmacologist with expertise in analyzing medication regimens and their effects.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

lab_test_agent = Agent(
    role="Lab Test Analyst",
    goal="Analyze and interpret all laboratory tests the patient has undergone",
    backstory="You are an expert in analyzing laboratory test codes and results. Your summaries are renowned for their depth, clarity, and insightful interpretation of lab test patterns and their clinical significance.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [13]:
# Define the prompt for basic member information retrieval
BASIC_INFO_PROMPT = """
Retrieve and display the following information for the member:
1. Member ID: {member_id}
2. Age: {age}
3. Gender: {gender}


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.
"""

In [14]:
DIAG_SUMMARY_PROMPT = """
Task Description
Your task is to summarize the following 'Patient Diagnosis Code History' and nothing else. The summary should be thorough, capturing ALL diagnoses present in the data.

Input Data
Patient Diagnosis Code History:
{diagnosis_history}

Output Format
Generate the summary in the following markdown format:

Diagnosis Summary:

Diagnosis Category 1:
- Diagnosis 2 (code2) in [dos1, dos2, dos3]
- Diagnosis 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Diagnosis Category 2:
- Diagnosis 6 (code6) in [dos1, dos2, dos3]
- Diagnosis 4 (code4) in [dos1]

Diagnosis Category 3: 
- Diagnosis 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Diagnosis 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1.Only include diagnoses and dates explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each diagnosis should appear only once, with all associated dates listed chronologically.
4.Prioritize more serious and then recent disease categories to the top.
5.Within each category, prioritize more serious and recent diseases to the top.
6.If a disease does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL diagnoses codes from the input data are included in the summary wihtout missing any.

Special Case
If there are no diagnoses in the patient's history, output only:
"There are no diagnoses observed in the patient's history."

Diagnosis Summary:
""".strip()

In [15]:
PROCEDURE_PROMPT = """

Task Description
Your task is to summarize the following 'Patient Procedure Code History' and nothing else. The summary should capture ALL procedures in the data.

Input Data
Patient Procedure Code History:
{procedure_history}

Output Format
Generate the summary in the following markdown format:

Procedure Summary:

Procedure Category 1:
- Proc 2 (code2) in [dos1, dos2, dos3]
- Proc 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Procedure Category 2:
- Proc 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Proc 4 (code4) in [dos1, dos2, dos3]

Procedure Category 3: 
- Proc 3 (code3) in [dos1, dos2, dos3]
- Proc 5 (code5) in [dos1, dos2]

Rules and Guidelines

1.Only include procedures and dates explicitly present in the provided input data.
2.Do not infer or add any information not directly stated in the input data.
3.Each procedure should appear only once, with all associated dates listed chronologically.
4.Prioritize more serious and then recent procedure categories to the top.
5.Within each category, prioritize more serious and recent procedures to the top.
6.If a procedure does not fit into a designated category, list it as a stand-alone item.
7.Ensure ALL procedures codes from the input data are included in the summary without missing any.

Special Case
If there are no procedures in the patient's history, output only:
"There were no procedures performed in the patient's history."

Procedure Summary:
""".strip()


In [16]:
MEDICATION_PROMPT = """
Task Description
Your task is to summarize the following 'Patient RX Code History' and nothing else. The summary should account for ALL medications in the data.

Input Data
Patient RX Code History:
{medication_history}

Output Format
Generate the summary in the following markdown format:

Prescription Medication Summary:

Medication Category 1:
- Rx 2 (code2) in [dos1, dos2, dos3]
- Rx 1 (code1) in [dos1, dos2, dos3, dos4]

Medication Category 2:
- Rx 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Rx 4 (code4) in [dos1, dos2, dos3]

Medication Category 3: 
- Rx 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Rx 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1.Categorize medications based on type of medication
2.Only include medications and dates explicitly present in the provided input data.
3.Do not infer or add any information not directly stated in the input data.
4.Each medication should appear only once, with all associated dates listed chronologically.
5.Prioritize more serious and then recent medication categories to the top.
6.Within each category, prioritize more serious and recent medications to the top.
7.If a medication does not fit into a designated category, list it as a stand-alone item.
8.Ensure ALL medications codes from the input data are included in the summary without missing any.

Special Case
If there are no medications in the patient's history, output only:
"There were no medications prescribed in the patient's history."


Prescription Medication Summary :
""".strip()


In [17]:
LAB_TEST_PROMPT ="""
Task Description
Your task is to summarize the following 'Patient Lab Code History' and nothing else. The summary should capture ALL lab tests in the data. For each test, you must display the exact abnormal code value from the input data.

Input Data
Patient Lab Code History:
{lab_test_history}

Output Format
Generate the summary in the following markdown format:
Lab Summary:

Lab Category 1:
- Complete Lab Test Name (code) : [date1:value1 abnormal_code_value, date2:value2 abnormal_code_value]

Lab Category 2:
- Complete Lab Test Name (code) : [date1:value1 abnormal_code_value, date2:value2 abnormal_code_value]

Rules and Guidelines
- Categorize lab tests into appropriate groups based on test type
- Include ONLY lab tests explicitly present in the input data
- Do not infer or add any information not directly stated
- If a value is missing, display nothing for value
- Use the exact abnormal_code value from input data (do not transform/map to other strings)
- Display all dates and results for each lab test in chronological order within square brackets
- Separate multiple date-value pairs with commas
- Each lab test should appear only once, consolidating all dates and results
- Prioritize more serious and then recent lab categories to the top
- Within each category, prioritize more serious and recent lab tests to the top
- List tests without a category as stand-alone items
- ALL lab tests from input data must be included without missing any
- No lab test should be omitted
- No duplicate entries allowed

Special Case
If there are no lab tests in the patient's history, output only:
"There were no lab tests conducted in the patient's history."

Lab Summary:
""".strip()

In [18]:
# Tasks for each agent
# Create a task for basic member information retrieval
basic_info_task = Task(
    description=BASIC_INFO_PROMPT,
    agent=basic_info_agent,
    expected_output="A concise display of the member's ID, age, and gender."
)

diagnosis_task = Task(
    description=DIAG_SUMMARY_PROMPT,
    agent=diagnosis_agent,
    expected_output="A comprehensive analysis of the patient's diagnostic history, or a statement indicating no diagnoses if applicable"
)

procedure_task = Task(
    description=PROCEDURE_PROMPT,
    agent=procedure_agent,
    expected_output="A detailed summary of all medical procedures the patient has undergone, or a statement indicating no procedures if applicable"
)

medication_task = Task(
    description=MEDICATION_PROMPT,
    agent=medication_agent,
    expected_output="A comprehensive summary of the patient's medication history, or a statement indicating no medications if applicable"
)

lab_test_task = Task(
    description=LAB_TEST_PROMPT,
    agent=lab_test_agent,
    expected_output="A detailed analysis of all lab tests with results the patient has undergone, or a statement indicating no lab tests if applicable"
)


In [34]:
past_condition_agent = Agent(role="Medical History Analyst",
                             goal="Identify and prioritize past medical conditions with detailed evidence.", 
                             verbose=True,
                             allow_delegation=False,
                             llm=llm,
                             backstory=("As a medical analyst, you specialize in examining patient medical history to identify past medical conditions,prioritize them, and provide detailed evidence.Your output should be professional and structured." )
                            )

past_condition_task = Task(
    description=("Analyze given {patient_medical_history} to identify past medical conditions.\n" 
                 "1. Generate a prioritized list of past medical conditions.\n" 
                     " - Prioritize based on severity, frequency, and long-term impact.\n" 
                 "2. For each condition, provide:\n" 
                     "a) Condition name and classification.\n" 
                     "b) Evidence, including diagnostic codes, lab results, procedures, and medications.\n" 
                     "c) A detailed timeline (if available) for diagnostic codes, lab results, procedures, and medications.\n" 
                     " d) A concise explanation of the condition's progression and impact.\n" ), 
    expected_output=
    ("A structured report including:\n" 
        "1. **Prioritized List of Past Medical Conditions**\n"
        "2. **Detailed Analysis for Each Condition**\n" 
             " a) Condition name and classification.\n" 
             " b) Evidence: diagnostic codes, lab tests, procedures, and medications.\n" 
             " c) Timeline: key dates for diagnosis codes, lab tests, procedures, and medications.\n" 
             " d) Impact assessment: severity and progression of the condition." 
       
    ),
    agent=past_condition_agent
)

In [20]:
future_condition_agent = Agent(
    role="Future Medical Condition Predictor",
    goal="Analyze and predict potential future medical conditions based on patient's current health status, medical history, risk factors, genetic predispositions, and comprehensive health data.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As an advanced medical predictive analyst, you specialize in forecasting potential future health risks "
        "and medical conditions. Your expertise lies in synthesizing complex medical data, including patient history, "
        "genetic information, lifestyle factors, and current health metrics to develop evidence-based predictive "
        "models. You excel at identifying early warning signs, potential disease trajectories, and personalized "
        "health risk assessments with a focus on preventive interventions and proactive health management."
    )
)

future_condition_task = Task(
    description=(
        "1. Review comprehensive patient data from previous agents (basic info, diagnosis, procedure, medication, lab test, and past condition agents).\n"
        "2. Conduct a comprehensive risk assessment for potential future medical conditions:\n"
        "   a. Analyze genetic predispositions\n"
        "   b. Evaluate lifestyle and environmental risk factors\n"
        "   c. Assess current health status and existing medical conditions\n"
        "   d. Review family medical history\n"
        "3. Structure the output as follows:\n"
        "   a. 'Prioritized List of Potential Future Medical Conditions'\n"
        "   b. Add a note: 'Predictions based on comprehensive risk analysis and current health data.'\n"
        "   c. Develop a 'Predictive Health Risk Summary' (around 250 words)\n"
        "   d. Create a 'Detailed Predictive Analysis' section for each potential condition\n"
        "4. For each potential future condition:\n"
        "   a. Clearly state the predicted medical condition\n"
        "   b. Provide a comprehensive risk assessment including:\n"
        "      - Probability of condition development (percentage)\n"
        "      - Estimated timeframe for potential onset\n"
        "      - Primary and secondary risk factors\n"
        "   c. Detail the evidence supporting the prediction:\n"
        "      - Relevant genetic markers\n"
        "      - Specific health indicators\n"
        "      - Lifestyle and environmental risk factors\n"
        "   d. Recommend preventive measures:\n"
        "      - Lifestyle modifications\n"
        "      - Potential screening tests\n"
        "      - Proactive medical interventions\n"
        "   e. Provide confidence level for each prediction\n"
        "5. Develop a comprehensive 'Preventive Action Plan':\n"
        "   a. Prioritized list of recommended preventive actions\n"
        "   b. Detailed rationale for each recommendation\n"
        "   c. Estimated impact on reducing future health risks\n"
        "6. Include potential impact on quality of life and long-term health trajectory\n"
        "7. If specific predictions cannot be made with high confidence, clearly state the limitations\n"
        "8. Ensure all predictions are supported by:\n"
        "   - Current medical research\n"
        "   - Statistical evidence\n"
        "   - Patient-specific data\n"
        "   - Recognized medical prediction models"
    ),
    expected_output=(
        "A comprehensive predictive analysis of potential future medical conditions, structured as follows:\n"
        "1. Prioritized List of Potential Future Medical Conditions\n"
        "2. Note on prediction basis\n"
        "3. Predictive Health Risk Summary (250 words)\n"
        "4. Detailed Predictive Analysis for Each Potential Condition:\n"
        "   - Predicted condition\n"
        "   - Probability of development\n"
        "   - Estimated onset timeframe\n"
        "   - Primary and secondary risk factors\n"
        "   - Supporting evidence (genetic markers, health indicators)\n"
        "   - Recommended preventive measures\n"
        "   - Confidence level of prediction\n"
        "5. Comprehensive Preventive Action Plan:\n"
        "   - Prioritized preventive actions\n"
        "   - Rationale for recommendations\n"
        "   - Estimated risk reduction impact\n"
        "6. Potential impact on long-term health and quality of life\n"
        "Note: Predictions based on current medical data, research, and patient-specific information. "
        "Limitations in prediction confidence will be clearly stated."
    ),
    agent=future_condition_agent,
    context=[basic_info_task, diagnosis_task, procedure_task, medication_task, lab_test_task, past_condition_task]
)

In [21]:
question_answer_generator_agent = Agent(
    role="Medical Question and Answer Generator",
    goal="Generate three important questions with answers to help patients discuss their condition with doctors",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an experienced medical professional with expertise in patient education and doctor-patient communication. "
        "Your role is to generate insightful questions and provide informative answers that will help patients have more "
        "productive discussions with their doctors about their condition, treatment options, and overall health management."
    )
)

question_answer_generator_task = Task(
    description=(
        "Based on the patient's medical information, generate three important questions with answers to help the patient "
        "discuss their condition with their doctor. Follow these steps:\n"
        "1. Review the patient's diagnosis history, lab results, medication history, procedure history, "
        "past medical conditions, and potential future conditions.\n"
        "2. Identify the most critical aspects of the patient's health status and medical history.\n"
        "3. Formulate three specific, targeted questions that address:\n"
        "   a. Understanding their current condition and its implications\n"
        "   b. Treatment options or management strategies\n"
        "   c. Preventive measures or lifestyle changes\n"
        "4. For each question, provide a clear and informative answer based on the patient's specific case and general medical knowledge.\n"
        "5. Ensure each question and answer pair is relevant to the patient's case and would provide valuable information for a doctor visit.\n"
        "6. Format the output as a markdown document with the following structure:\n"
        "   - Start with the title '# Important Questions to Discuss with Your Doctor'\n"
        "   - Include the note about LLM-generated content\n"
        "   - Follow the specified question-answer-importance format"
    ),
    expected_output=(
        "A comprehensive markdown-formatted document that includes:\n"
        "1. Title: '# Important Questions to Discuss with Your Doctor'\n"
        "2. Disclaimer Note\n"
        "3. Three detailed question-answer pairs with importance explanations, specifically tailored to the patient's medical profile\n"
        "4. Each section clearly formatted with markdown headers and paragraphs"
    ),
    agent=question_answer_generator_agent,
    context=[diagnosis_task, lab_test_task, medication_task, procedure_task, past_condition_task, future_condition_task]
)

In [22]:
HOS_AGENT = Agent(
role="HOS Analyst",
goal="Provide a comprehensive summary of discussions related to the HOS fall risk management measure in the provided EMR notes",
backstory="You are an expert in healthcare quality measures, with a specific focus on the HOS and fall risk management for older adult patients",
verbose=True,
allow_delegation=False,
llm=llm
)

In [23]:
HOS_SUMMARY_PROMPT = """
Task Description
Your task is to analyze the provided Electronic Medical Record (EMR) notes and summarize any discussions related to the Health Outcomes Survey (HOS) fall risk management measure.

Input Data
EMR Notes:
{emr_notes_history}

Output Format
Provide a summary in the following format:
HOS Fall Risk Management Summary:
[If no relevant discussions found]
There are no discussions related to the HOS fall risk management measure observed in the provided EMR notes.
[If relevant discussions found]
The EMR notes contain the following discussions related to the HOS fall risk management measure:
Discussion 1:
Question: [Summarize the key question or topic discussed]
Answer: [Summarize the key points or responses related to the question/topic]
Discussion 2:
Question: [Summarize the key question or topic discussed]
Answer: [Summarize the key points or responses related to the question/topic]
[Add more discussions as needed]

Rules and Guidelines

Thoroughly review the EMR notes to identify any mentions of the HOS fall risk management measure or related discussions.
Summarize the key questions or topics discussed, as well as the relevant answers or responses.
If no discussions related to the HOS measure are found, clearly state that there are no relevant discussions observed.
Ensure the summary is concise, informative, and captures all the important details from the EMR notes.

HOS Measure Summary :
""".strip()


HOS_SUMMARY_TASK = Task(
description=HOS_SUMMARY_PROMPT,
agent=HOS_AGENT,
expected_output="A summary of discussions related to the HOS fall risk management measure, or a statement indicating no relevant discussions were found."
)

In [24]:
import matplotlib.pyplot as plt
import io
from PIL import Image
from typing import List, Dict, Union

import os
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List, Dict, Union

@tool
def plot_lab_test(test_name: str, data: List[Dict[str, Union[str, float]]], config: dict = None):
    """
    Generate a graph of lab test results over time and return a filename.
    
    Parameters:
    test_name (str): The name of the lab test, used for the graph title and y-axis label.
    data (List[Dict[str, Union[str,str]]]): A list of dictionaries, where each dictionary
        contains 'date' (str) and 'value' (float) keys representing a single test result.
    config (dict, optional): Configuration options for plot customization.
    
    Returns:
    str: Filename of the saved plot
    """
    # Set default configuration
    default_config = {
        'figsize': (10, 6),
        'marker': 'o',
        'rotation': 45,
        'color': 'blue',
        'line_style': '-'
    }
    
    # Override default config with user-provided config
    if config is not None:
        default_config.update(config)
    
    # Extract dates and values
    dates = [point['date'] for point in data]
    values = [point['value'] for point in data]
    
    # Create the plot
    plt.figure(figsize=default_config['figsize'])
    plt.plot(
        dates, 
        values, 
        marker=default_config['marker'], 
        color=default_config['color'],
        linestyle=default_config['line_style']
    )
    plt.title(f'{test_name} Test Results Over Time')
    plt.xlabel('Date')
    plt.ylabel(f'{test_name} Value')
    plt.xticks(rotation=default_config['rotation'])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Create a directory to save plots if it doesn't exist
    os.makedirs('saved_plots', exist_ok=True)
    
    # Generate a unique filename using the test name and current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join('saved_plots', f"{test_name.replace(' ', '_')}_{timestamp}.png")
    
    # Save the plot as an image file
    plt.savefig(filename)
    
    # Clear the current figure to free up memory
    plt.clf()
    plt.close()
    
    return filename

In [25]:
# Define the Intelligent Medical Chatbot Agent
intelligent_medical_chatbot = Agent(
    role="Smart Medical Expert",
    goal="Provide accurate and helpful responses to user questions about medical conditions, interpret medical codes, and generate visualizations of lab test results when requested",
    tools=[search_tool, scrape_tool, plot_lab_test],  
    llm=llm, 
    backstory=(
        "You are an AI-powered medical expert chatbot with extensive knowledge of medical codes and the ability to analyze and interpret medical records. "
        "You can answer both patient-specific and general medical queries, leveraging your access to patient information including basic details, diagnosis history, "
        "procedure history, medication history, lab test results, past medical conditions, and potential future conditions. "
        "You are allowed to make multiple calls (either together or in sequence). "
        "You're equipped with a lab visualization tool for generating graphs of lab test results. "
        "Your responses are based on your comprehensive medical knowledge and the patient data provided."
    )
)

intelligent_medical_chatbot_task = Task(
    description=(
        "As a smart medical expert, your task is to answer '{user_question}' accurately and helpfully. Follow these guidelines:\n"
        "1. Analyze the user's question to determine the type of information needed.\n"
        "2. For patient-specific questions:\n"
        "   - Review and use the provided patient data.\n"
        "   - Use patient-specific information to craft a precise response.\n"
        "3. For general medical queries or questions not related to a specific patient:\n"
        "   - Utilize the search_tool or scrape_tool to gather relevant and up-to-date medical information.\n"
        "   - Cross-reference multiple sources to ensure accuracy.\n"
        "   - Synthesize the gathered information into a comprehensive response.\n"
        "4. Use your extensive medical knowledge to interpret and explain medical information.\n"
        "5. Interpret medical codes using your expertise when required.\n"
        "6. For lab test visualization:\n"
        "   - If explicitly requested or if it would enhance your explanation, use the plot_lab_test tool.\n"
        "   - Provide the exact lab_code name when calling the tool.\n"
        "   - The tool will return a filename (e.g., 'saved_plots/lab_code_timestamp.png').\n"
        "   - Include this filename in your response text.\n"
        "   - Do not attempt to embed the image yourself; the chat function will handle this.\n"
        "7. You can use the plot_lab_test tool multiple times if needed for different lab tests.\n"
        "8. Format your response clearly:\n"
        "   - Provide a direct answer to the question.\n"
        "   - Include relevant details from patient information, web search, or your medical knowledge.\n"
        "   - If you've generated a graph, mention the filename in your text response.\n"
        "   - Provide a brief interpretation of the graph.\n"
        "9. If the question is outside your scope or requires immediate medical attention, advise consulting a healthcare professional.\n"
        "10. Maintain patient confidentiality at all times.\n"
    ),
    agent=intelligent_medical_chatbot,
    context=[basic_info_task, diagnosis_task, procedure_task, medication_task, lab_test_task, past_condition_task, future_condition_task],
    expected_output=(
        "A clear, informative, and accurate response addressing the user's question. "
        "For patient-specific queries, reference the sources of patient information. "
        "For general queries, include sources from web searches. "
        "If a graph was generated, the filename should be mentioned in the text response. "
        "The response should combine textual explanation with references to generated graphs when appropriate."
    )
)

In [26]:
df.columns

Index(['id', 'dos', 'type', 'code', 'value', 'abnormal_cd', 'cd_desc'], dtype='object')

In [27]:
def process_member_data(member_id):
    df_member = df[df['id'] == member_id]
    
    if df_member.empty:
        return None, None, None, None
    
    #extract basic information 
    df_age_gender=df_member[(df_member['type']=='age') | (df_member['type']=='gender')].reset_index(drop=True)
    age=df_age_gender.loc[0,'value']
    gender=df_age_gender.loc[1,'value']
    
    # age=df_member['age'][0]
    # gender=df_member['gender'][0]
    
    df_diag = df_member[df_member['type'] == 'diag']
    df_diag = df_diag[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_proc = df_member[df_member['type'] == 'proc']
    df_proc = df_proc[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_rx = df_member[df_member['type'] == 'gpi']
    df_rx = df_rx[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_lab = df_member[df_member['type'] == 'lab']
    df_lab['value'] = pd.to_numeric(df_lab['value'], errors='coerce')
    df_lab['value']= np.round(df_lab['value'],2)
    df_lab=df_lab[['id','dos','code','value','abnormal_cd','cd_desc','type']]
    df_emr = df_member[df_member['type'] == 'EMR_Note']
    df_emr = df_emr[['id','dos','code','value','cd_desc','type']]
    
    basic_info_crew = Crew(agents=[basic_info_agent], tasks=[basic_info_task], verbose=True)
    basic_info_result = basic_info_crew.kickoff({"member_id": member_id,"age":age,"gender":gender})
    
    diagnosis_crew = Crew(agents=[diagnosis_agent], tasks=[diagnosis_task], verbose=True)
    diag_result = diagnosis_crew.kickoff({"diagnosis_history": df_diag.to_string()})
    
    procedure_crew = Crew(agents=[procedure_agent], tasks=[procedure_task], verbose=True)
    proc_result = procedure_crew.kickoff({"procedure_history": df_proc.to_string()})
    
    medication_crew = Crew(agents=[medication_agent], tasks=[medication_task], verbose=True)
    med_result = medication_crew.kickoff({"medication_history": df_rx.to_string()})
    
    lab_test_crew = Crew(agents=[lab_test_agent], tasks=[lab_test_task], verbose=True)
    lab_result = lab_test_crew.kickoff({"lab_test_history": df_lab.to_string()})
    
    emr_notes_crew = Crew(agents=[HOS_AGENT], tasks=[HOS_SUMMARY_TASK], verbose=True)
    emr_notes_result = emr_notes_crew.kickoff({"emr_notes_history": df_emr.to_string()})
    
    past_condition_crew = Crew(agents=[past_condition_agent], tasks=[past_condition_task], verbose=True)
    past_condition_result = past_condition_crew.kickoff()
    
    future_condition_crew = Crew(agents=[future_condition_agent], tasks=[future_condition_task], verbose=True)
    future_condition_result = future_condition_crew.kickoff()
    
    question_answer_generator_crew = Crew(agents=[question_answer_generator_agent], tasks=[question_answer_generator_task], verbose=True)
    question_answer_generator_result = question_answer_generator_crew.kickoff()
    
    return basic_info_result,diag_result, proc_result, med_result, lab_result,emr_notes_result,past_condition_result,future_condition_result,question_answer_generator_result

In [28]:
def gradio_interface(member_id): 
    try:
        member_id = int(member_id) 
    except ValueError:
        return "Invalid member ID. Please enter a number.", "", "", "", "", "", "", "", "", "",""
    
    # Process the member data and get results from other agents 
    results = process_member_data(member_id) 
    if results[0] is None:
        return "No data found for this member ID.", "", "", "", "", "", "", "", "", "",""
    
    return results

In [29]:
import re
import base64
from PIL import Image
import io

def chat(message, history):
    med_chatbot_crew = Crew(agents=[intelligent_medical_chatbot], tasks=[intelligent_medical_chatbot_task], verbose=True)
    
    response = med_chatbot_crew.kickoff({"user_question": message, "History": history})
    
    # Use regex to find all .png filenames in the saved_plots directory
    image_paths = re.findall(r'saved_plots/[^()]+\([^)]+\)_\d+_\d+\.png', response)
    
    if image_paths:
        # Get the last image path (in case there are multiple)
        print(image_paths)
        image_path = image_paths[-1]
        print(f"Image path: {image_path}")
        
        # Extract the text response by removing the image path
        #text_response = response.replace(image_path, '').strip()
        
        try:
            with Image.open(image_path) as img:
                buffer = io.BytesIO()
                img.save(buffer, format="PNG")
                encoded_image = base64.b64encode(buffer.getvalue()).decode()
            
            full_response = f"{response}\n\nHere's the requested graph:\n\n![Lab Test Graph](data:image/png;base64,{encoded_image})"
            
            history.append((message, full_response))
        except Exception as e:
            error_response = f"An error occurred while processing the graph: {str(e)}"
            history.append((message, f"{response}\n\n{error_response}"))
    else:
        history.append((message, response))
    
    return "", history

In [30]:
def save_results(member_id, basic_info, diag, proc, med, lab,hos, past_condition, future_condition, qa):
    results = {
        "Member ID": member_id,
        "Basic Info": basic_info,
        "Diagnosis Summary": diag,
        "Procedure Summary": proc,
        "Medication Summary": med,
        "Lab Summary": lab,
        "HOS Measure Summary":hos,
        "Past Medical Conditions": past_condition,
        "Potential Future Conditions": future_condition,
        "Common Q&A": qa
    }
    
    filename = f"results_for_patient_ID{member_id}.md"
    with open(filename, 'w') as f:
        f.write(f"# Medical Record Results for Member ID: {member_id}\n\n")
        for key, value in results.items():
            f.write(f"## {key}\n\n{value}\n\n")
    
    return f"Results saved to {filename}"

In [31]:
import gradio as gr

css = """
h1 {
    text-align: center;
    display:block;
}
"""

def create_patient_interface(num_patients):
    with gr.Blocks(fill_height=True, fill_width=True, css=css, theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.sky)) as demo:
        gr.Markdown("# Chat With Medical Records")
        gr.Markdown("Enter a member ID to get a detailed analysis of the patient's medical history.")
        
        with gr.Column(scale=6):
            with gr.Tabs() as main_tabs:
                # Lists to store components
                member_id_inputs = []
                submit_buttons = []
                basic_info_outputs = []
                diag_outputs = []
                proc_outputs = []
                med_outputs = []
                lab_outputs = []
                hos_outputs =[]
                past_condition_outputs = []
                future_condition_outputs = []
                question_answer_outputs = []
                save_buttons = []
                save_outputs = []

                # Create a tab for each patient
                for i in range(1, num_patients + 1):
                    with gr.Tab(f'Patient {i}'):
                        with gr.Column():
                            # Member ID Input and Submit
                            with gr.Row():
                                member_id_input = gr.Textbox(label=f"Enter Member ID for Patient {i}")
                                submit_button = gr.Button("Submit")
                                member_id_inputs.append(member_id_input)
                                submit_buttons.append(submit_button)

                            # Results Tabs
                            with gr.Tabs() as result_tab:
                                with gr.TabItem("Member Info"):
                                    basic_info_output = gr.Markdown()
                                    basic_info_outputs.append(basic_info_output)

                                with gr.TabItem("Diagnosis Summary"):
                                    diag_output = gr.Markdown()
                                    diag_outputs.append(diag_output)

                                with gr.TabItem("Procedure Summary"):
                                    proc_output = gr.Markdown()
                                    proc_outputs.append(proc_output)

                                with gr.TabItem("Medication Summary"):
                                    med_output = gr.Markdown()
                                    med_outputs.append(med_output)

                                with gr.TabItem("Lab Summary"):
                                    lab_output = gr.Markdown()
                                    lab_outputs.append(lab_output)
                                    
                                with gr.TabItem("HOS Measure Summary"):
                                    hos_output = gr.Markdown()
                                    hos_outputs.append(hos_output)

                                with gr.TabItem("Past Medical Conditions"):
                                    past_condition_output = gr.Markdown()
                                    past_condition_outputs.append(past_condition_output)

                                with gr.TabItem("Potential Future Conditions"):
                                    future_condition_output = gr.Markdown()
                                    future_condition_outputs.append(future_condition_output)

                                with gr.TabItem("Ask doctor"):
                                    question_answer_output = gr.Markdown()
                                    question_answer_outputs.append(question_answer_output)


                            # Save Results
                            with gr.Row():
                                save_button = gr.Button("Save Results")
                                save_output = gr.Markdown()
                                save_buttons.append(save_button)
                                save_outputs.append(save_output)

            # Medical Chatbot Section - Outside of Patient Tabs
            #gr.Markdown("## Medical Chatbot")
            with gr.TabItem("Medical Chatbot"):
                chatbot = gr.Chatbot(label="Medical Chatbot", height=600)
                msg = gr.Textbox(label="Ask a question")
                clear = gr.Button("Clear Chat")

            # Event Handlers
            for i in range(num_patients):
                # Submit button event
                submit_buttons[i].click(
                    fn=gradio_interface,  
                    inputs=member_id_inputs[i],
                    outputs=[
                        basic_info_outputs[i], diag_outputs[i], 
                        proc_outputs[i], med_outputs[i], 
                        lab_outputs[i], hos_outputs[i],past_condition_outputs[i], 
                        future_condition_outputs[i], 
                        question_answer_outputs[i]
                    ]
                )

                # Save button event
                save_buttons[i].click(
                    fn=save_results, 
                    inputs=[
                        member_id_inputs[i], 
                        basic_info_outputs[i], diag_outputs[i], 
                        proc_outputs[i], med_outputs[i], 
                        lab_outputs[i], hos_outputs[i],past_condition_outputs[i], 
                        future_condition_outputs[i], 
                        question_answer_outputs[i]
                    ],
                    outputs=save_outputs[i]
                )

            # Chatbot events
            msg.submit(
                fn=chat,  
                inputs=[msg, chatbot], 
                outputs=[msg, chatbot]
            )

            clear.click(
                lambda: None, 
                None, 
                chatbot, 
                queue=False
            )

        return demo

# Create interface with 3 patients
demo = create_patient_interface(1)
demo.launch(share=True)

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
which: no node in (/home/jovyan/envs/llm_agents/bin:/miniconda3/condabin:/opt/mssql-tools/bin:/usr/local/apache-jmeter-5.4.3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/miniconda3/bin:/usr/lib/spark:/usr/lib/spark/bin:/usr/lib/jvm/java-1.8.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://65887ea47394601d10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/jovyan/tmp/ipykernel_172308/1294541970.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab['value'] = pd.to_numeric(df_lab['value'], errors='coerce')
/home/jovyan/tmp/ipykernel_172308/1294541970.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab['value']= np.round(df_lab['value'],2)


 [2024-11-26 13:54:49][DEBUG]: == Working Agent: Basic Member Information Provider
 [2024-11-26 13:54:49][INFO]: == Starting Task: 
Retrieve and display the following information for the member:
1. Member ID: 28329
2. Age: 74.0
3. Gender: 0.0


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.



> Entering new CrewAgentExecutor chain...


2024-11-26 13:54:51,068 - 140103800301312 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


...

Thought: I now can give a great answer  
Final Answer: The EMR notes contain the following discussions related to the HOS fall risk management measure:

Discussion 1:  
Question: Does the patient have a history of fall in the last 12 months?  
Answer: Yes, the patient has a history of fall in the last 12 months, but was not injured because of the fall. Education was given to the patient on preventing falls.

Discussion 2:  
Question: Is the patient able to do the following activities of daily living on their own?  
Answer: The patient is independent in activities of daily living such as bathing, dressing, eating, using the toilet, transferring, continence, grocery shopping, laundry, preparing meals, housework, managing medication, and transportation.

These discussions highlight the assessment and management of fall risk in the patient, aligning with the HOS fall risk management measure.

> Finished chain.
 [2024-11-26 13:58:10][DEBUG]: == [HOS Analyst] Task output: The EMR notes conta

2024-11-26 13:58:35,768 - 140103800301312 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


To complete the task, I need to analyze the provided medical history data and structure it according to the specified format. The data includes diagnosis, procedure, medication, and lab test summaries. I will now proceed to organize and analyze this information to create a comprehensive report on the patient's past medical conditions.

Final Answer:

1. Prioritized List of Past Medical Conditions:
   - Type 2 Diabetes Mellitus with Complications
   - Hypertension and Cardiovascular Issues
   - Chronic Kidney Disease
   - Musculoskeletal Disorders
   - Gastrointestinal Disorders
   - Respiratory Conditions
   - Neurological and Psychological Conditions
   - Nutritional and Metabolic Disorders
   - Hematological Conditions
   - Endocrine Disorders
   - Vision Issues
   - Injuries

2. Note: See below for a summary and detailed analysis of each condition.

3. Summary of Past Health Issues:
The patient, a 74-year-old female, has a complex medical history with multiple chronic conditions. Th

2024-11-26 13:59:22,481 - 140103800301312 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


Thought: I now can give a great answer
Final Answer: 

1. **Prioritized List of Potential Future Medical Conditions:**
   - Cardiovascular Disease Progression
   - Advanced Chronic Kidney Disease
   - Osteoporosis with Fracture Risk
   - Cognitive Decline and Dementia Progression
   - Respiratory Complications from COPD

2. **Note:** Predictions based on comprehensive risk analysis and current health data.

3. **Predictive Health Risk Summary:**
   The patient, a 74-year-old female, presents with a complex medical history that includes Type 2 Diabetes Mellitus with complications, hypertension, chronic kidney disease, and musculoskeletal disorders. Given her current health status and medical history, there is a significant risk for the progression of cardiovascular disease, particularly due to her hypertension and hyperlipidemia. The presence of chronic kidney disease suggests a potential for further renal function decline, necessitating vigilant monitoring. Osteoporosis, coupled with a